# **Modeling Notebook**

### In this notebook we will pick up where we left off and start with our next model(s) for evaluation, keeping track of them through our metrics table up top.  Once we finish all of our models we will select the ones to continue with the model stack and then on to the Bayes Optimization.

#### First as usual let's read in our dataframe for usage and train our X,y values for our models to come.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [2]:
df_scaled = pd.read_csv(f'/Users/ryanm/Desktop/df-scaled.csv')
print(df_scaled.shape)
df_scaled.head(5)

(3214874, 24)


,user_id,order_number,order_id,order_dow,order_hour_of_day,product_id,reordered,aisle_id,department_id,product_name_code,...,user_product_order_count,avg_cart_position,product_popularity,user_reorder_proportion,total_user_unique_products,total_orders,days_since_prior_order,add_to_cart_order,avg_days_between_orders,std_days_between_orders
0,1,1,2539329,3,9,26405,0,54,17,31683,...,-0.410997,-1.920683,-0.508987,0.146423,-0.76297,-0.901902,0.031628,-0.676228,1.461402,1.007456
1,1,2,2398795,4,8,26088,1,23,19,980,...,-0.551215,-1.689794,-0.513694,0.146423,-0.76297,-0.901902,0.493386,-0.543345,1.461402,1.007456
2,1,3,473747,4,13,30450,1,88,13,7124,...,-0.551215,-1.592202,-0.470523,0.146423,-0.76297,-0.901902,1.186023,-0.676228,1.461402,1.007456
3,1,4,2254736,5,8,26405,1,54,17,31683,...,-0.410997,-1.920683,-0.508987,0.146423,-0.76297,-0.901902,2.109539,-0.676228,1.461402,1.007456
4,1,5,431534,5,16,41787,1,24,4,2419,...,-0.551215,-0.666268,-0.352629,0.146423,-0.76297,-0.901902,1.994099,-0.277578,1.461402,1.007456


In [3]:
X = df_scaled.drop('reordered', axis = 1)
y = df_scaled['reordered']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
main_path = r'C:/Users/ryanm/Desktop/df-scaled.csv'

#df_mod = df
df_scaled.to_csv(main_path, index = False)